In [96]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings("ignore")

In [97]:
df = pd.read_csv("./datasets/tmdb_5000_movies.csv")
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [98]:
df["genres"][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [99]:
# 장르 전처리
from ast import literal_eval

df["genres"] = df["genres"].apply(literal_eval)
df["keywords"] = df["keywords"].apply(literal_eval)
df["genres"] = df["genres"].apply(lambda x : [y["name"] for y in x])
df["keywords"] = df["keywords"].apply(lambda x : [y["name"] for y in x])
df[["genres", "keywords"]][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


In [100]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
df["genres_literal"] = df["genres"].apply(lambda x : (" ").join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(df["genres_literal"])
print(genre_mat.shape)

(4803, 276)


In [101]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [102]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [103]:
# 특정 영화와 장르 유사도가 높은 영화를 반환하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n=10) :

    # 인자로 입력된 df DataFrame에서 "title" 컬럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df["title"] == title_name]

    # title_named을 가진 DataFrame 의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [104]:
similar_movies = find_sim_movie(df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title", "vote_average"]]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [105]:
# @ 가중평점(Weighted Ratin) :
#     (v/(v+m)) * R + (m/(v+m)) * C

# - v : 영화별 평점을 투표한 횟수(vote_count)
# - m : 평점을 부여하기 위한 최소 투표 횟수 -> 여기서는 투표수 상위 60%
# - R : 개별 영화에 대한 평균 평점(vote_average)
# - C : 전체 영화에 대한 평균 평점(df["vote_average"].mean())
# # C, m 은 고정값
# # v, R 은 영화마다 변동값

# 상위 60%에 해당하는 vote_count를 최소 투표 횟수인 m으로 지정
C = df["vote_average"].mean()
m = df["vote_count"].quantile(0.6)

# C : 전체 영화에 대한 평균 평점 = 약 6점
# m : 평점을 부여하기 위한 최소 투표 횟수 = 370회(상위 60% 수준)
print("C : ", round(C, 3), "m : ", round(m, 3))

C :  6.092 m :  370.2


In [106]:
def weighted_vote_average(record) :
    v = record["vote_count"]
    R = record["vote_average"]

    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

In [107]:
# 기존 데이터에 가중평점 칼럼 추가
df["weighted_vote"] = df.apply(weighted_vote_average, axis = 1)

In [108]:
def find_sim_movie_ver2(df, sorted_ind, title_name, top_n=10) :
    title_movie = df[df["title"] == title_name]
    title_index = title_movie.index.values

    # top_n 의 2배에 해당하는 장르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n 의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values("weighted_vote", ascending = False)[:top_n]

In [109]:
similar_movies = find_sim_movie_ver2(df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title", "vote_average", "weighted_vote", "genres", "vote_count"]]

,title,vote_average,weighted_vote,genres,vote_count
2731,The Godfather: Part II,8.3,8.079586,"[Drama, Crime]",3338
1847,GoodFellas,8.2,7.976937,"[Drama, Crime]",3128
3866,City of God,8.1,7.759693,"[Drama, Crime]",1814
1663,Once Upon a Time in America,8.2,7.657811,"[Drama, Crime]",1069
883,Catch Me If You Can,7.7,7.557097,"[Drama, Crime]",3795
281,American Gangster,7.4,7.141396,"[Drama, Crime]",1502
4041,This Is England,7.4,6.739664,"[Drama, Crime]",363
1149,American Hustle,6.8,6.717525,"[Drama, Crime]",2807
1243,Mean Streets,7.2,6.626569,"[Drama, Crime]",345
2839,Rounders,6.9,6.530427,"[Drama, Crime]",439
